In [1]:
import mm_predictor # stat-analysis library
from sklearn import cross_validation, linear_model
import csv
import random
import numpy
import pandas as pd
#from xgboost import XGBClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# intialize stat & elo dictionaries
mm_predictor.init()

In [3]:
# Load data
season_data = pd.read_csv('./ncaa-data/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data = tourney_data[tourney_data.Season != 2017]

In [4]:
aggregated_data = pd.concat([season_data, tourney_data])

X,Y = mm_predictor.analyze_teams(aggregated_data)
# Run script

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = \
#             train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:-0.5443811951009836
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
    RobustScaler(),
    StackingEstimator(estimator=GaussianNB()),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
    StackingEstimator(estimator=GaussianNB()),
    LogisticRegression(C=5.0, dual=False, penalty="l1")
)

exported_pipeline = LogisticRegression()

exported_pipeline.fit(np.array(X), np.array(Y))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [6]:
# perform simluation of 2017 NCAA tourney
print("Simulating 2017 NCAA tourney")
submission_data = []
tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
tourney_2017_teams = tourney_seeds[tourney_seeds['Season'] == 2017]['Team']

stat_fields = ['score', 'fga', 'fgp', 'fga3', '3pp', 'ftp', 'or', 'dr',
                   'ast', 'to', 'stl', 'blk', 'pf']
print("Predicting matchups.")
tourney_2017_teams.sort_values()

for team_1 in tourney_2017_teams:
    for team_2 in tourney_2017_teams:
        if team_1 != team_2:
            prediction = mm_predictor.predict_winner(
                team_1, team_2, exported_pipeline, 2017, stat_fields)
            label = '2017_' + str(team_1) + '_' + str(team_2)
            submission_data.append([label, prediction[0][0]])

Simulating 2017 NCAA tourney
Predicting matchups.


In [14]:
teamNames = pd.read_csv('./ncaa-data/Teams.csv')

team_id_map = {}
for index, row in teamNames.iterrows():
    team_id_map[row['TeamID']] = row['TeamName']

teamTeamWinChanceMap = {}
for line in submission_data:
    title = line[0].split('_')
    team1 = team_id_map[int(title[1])]
    team2 = team_id_map[int(title[2])]
    winRate = str(round(line[1]* 100,2))
    if team1 not in teamTeamWinChanceMap:
        teamTeamWinChanceMap[team1] = {}
    if float(winRate) >= 50:
        if team2 in teamTeamWinChanceMap and team1 in teamTeamWinChanceMap[team2] and teamTeamWinChanceMap[team2][team1] < float(winRate):
            # update teamTeammap[team2][team1] to remove that key
            teamTeamWinChanceMap[team2].pop(team1, None)
        teamTeamWinChanceMap[team1][team2] = float(winRate)
            
print(teamTeamWinChanceMap)

{'Villanova': {'Duke': 55.01, 'Baylor': 67.47, 'Florida': 66.18, 'Virginia': 69.19, 'South Carolina': 76.98, 'Wisconsin': 67.78, 'Virginia Tech': 70.56, 'Marquette': 76.33, 'Providence': 73.05, 'USC': 73.19, 'UNC Wilmington': 60.48, 'ETSU': 66.83, 'New Mexico St': 67.94, 'Troy': 76.59, "Mt St Mary's": 82.66, 'New Orleans': 81.71, 'Florida St': 65.2, 'West Virginia': 63.38, 'Notre Dame': 64.05, 'Maryland': 70.56, "St Mary's CA": 56.26, 'Northwestern': 73.05, 'Vanderbilt': 73.76, 'VA Commonwealth': 63.71, 'Xavier': 78.11, 'Princeton': 63.38, 'Bucknell': 68.88, 'FL Gulf Coast': 70.26, 'North Dakota': 76.72, 'S Dakota St': 81.92, 'Kansas': 51.4, 'Louisville': 63.54, 'Oregon': 53.03, 'Purdue': 61.85, 'Iowa St': 63.04, 'Creighton': 69.95, 'Michigan': 63.88, 'Miami FL': 71.01, 'Michigan St': 78.24, 'Oklahoma St': 74.86, 'Rhode Island': 62.7, 'Nevada': 58.73, 'Vermont': 56.08, 'Iona': 76.85, 'Jacksonville St': 82.76, 'NC Central': 78.85, 'UC Davis': 80.37, 'North Carolina': 57.68, 'UCLA': 52.4

In [ ]:
print("Writing %d results." % len(submission_data))
with open('./ncaa-data/submission.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'pred'])
    writer.writerows(submission_data)

In [15]:
tourney_data2017 = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data2017 = tourney_data2017[tourney_data2017.Season == 2017]

winners = []

tourney_data_grouped = tourney_data2017[4:].groupby('WTeamID').size().reset_index(name='NumWins')
for index, row in tourney_data_grouped.iterrows():
    teamName = team_id_map[row['WTeamID']]
    wins = row['NumWins']
    if len(winners) == 0:
        winners.append([])
    winners[0].append(teamName)
    if wins > 1:
        if len(winners) == 1:
            winners.append([])
        winners[1].append(teamName)
    if wins > 2:
        if len(winners) == 2:
            winners.append([])
        winners[2].append(teamName)
    if wins > 3:
        if len(winners) == 3:
            winners.append([])
        winners[3].append(teamName)
    if wins > 4:
        if len(winners) == 4:
            winners.append([])
        winners[4].append(teamName)
    if wins > 5:
        if len(winners) == 5:
            winners.append([])
        winners[5].append(teamName)

In [16]:
tourney_data2017

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
914,2017,134,1243,95,1448,88,N,0,31,47,...,27,29,32,7,14,14,11,8,1,25
915,2017,134,1291,67,1309,66,N,0,25,47,...,9,7,12,10,18,12,14,10,2,18
916,2017,135,1413,67,1300,63,N,0,22,48,...,26,10,13,9,21,14,9,10,0,15
917,2017,135,1425,75,1344,71,N,0,24,57,...,20,12,17,7,20,17,9,3,2,21
918,2017,136,1112,100,1315,82,N,0,36,62,...,22,8,16,9,22,18,6,6,2,23
919,2017,136,1139,76,1457,64,N,0,24,49,...,25,5,10,3,24,12,9,6,1,22
920,2017,136,1196,80,1190,65,N,0,26,54,...,24,6,10,13,19,12,17,9,0,19
921,2017,136,1199,86,1195,80,N,0,30,54,...,28,14,20,7,19,18,11,6,1,23
922,2017,136,1211,66,1355,46,N,0,25,63,...,20,5,11,9,29,8,13,3,4,17
923,2017,136,1235,84,1305,73,N,0,30,54,...,26,13,16,10,18,13,7,8,4,14


In [20]:
# First four rows is 'first four' and not in actual bracket
firstFour = tourney_data2017[:4]
mainTourney = tourney_data2017[4:]

tourney = [-1] * max(tourney_seeds['Team'])

# Look at first four teams
for index, row in firstFour.iterrows():
    index1 = int(row['WTeamID'])
    index2 = int(row['LTeamID'])
    team1Name = team_id_map[index1]
    team2Name = team_id_map[index2]
    
    if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]):
        tourney[index2] = index1
        tourney[index1] = tourney[index1] - 1
    else:
        tourney[index1] = index2
        tourney[index2] = tourney[index2] - 1

# MAIN TOURNAMENT 

tourney = [-1] * max(tourney_seeds['Team'])
score = 0
for index, row in mainTourney.iterrows():
    index1 = int(row['WTeamID'])
    index2 = int(row['LTeamID'])
    while tourney[index1] > 0:
        index1 = tourney[index1]
    while tourney[index2] > 0:
        index2 = tourney[index2]
    team1Name = team_id_map[index1]
    team2Name = team_id_map[index2]
    
    if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]): # team1 would win
        tourney[index2] = index1
        tourney[index1] = tourney[index1] - 1
        if team1Name in winners[abs(tourney[index1]) - 2]:
            score += (abs(tourney[index1]) - 1) * 10
            print('SCORE')
        print(team1Name + ' vs ' + team2Name + ', team 1 wins')
    else: #team2 would win
        tourney[index1] = index2
        tourney[index2] = tourney[index2] - 1
        if team2Name in winners[abs(tourney[index2]) - 2]:
            score += (abs(tourney[index2]) - 1) * 10
            print('SCORE')

        print(team1Name + ' vs ' + team2Name + ', team 2 wins')
print(score)

SCORE
Arizona vs North Dakota, team 1 wins
Butler vs Winthrop, team 2 wins
Florida vs ETSU, team 2 wins
SCORE
Florida St vs FL Gulf Coast, team 1 wins
SCORE
Gonzaga vs S Dakota St, team 1 wins
Iowa St vs Nevada, team 2 wins
SCORE
MTSU vs Minnesota, team 1 wins
Northwestern vs Vanderbilt, team 2 wins
Notre Dame vs Princeton, team 2 wins
Purdue vs Vermont, team 2 wins
SCORE
St Mary's CA vs VA Commonwealth, team 1 wins
SCORE
Villanova vs Mt St Mary's, team 1 wins
Virginia vs UNC Wilmington, team 2 wins
SCORE
West Virginia vs Bucknell, team 1 wins
Wisconsin vs Virginia Tech, team 2 wins
Xavier vs Maryland, team 2 wins
Arkansas vs Seton Hall, team 2 wins
Baylor vs New Mexico St, team 2 wins
SCORE
Cincinnati vs Kansas St, team 1 wins
SCORE
Duke vs Troy, team 1 wins
SCORE
Kansas vs UC Davis, team 1 wins
SCORE
Kentucky vs N Kentucky, team 1 wins
SCORE
Louisville vs Jacksonville St, team 1 wins
SCORE
Michigan vs Oklahoma St, team 1 wins
Michigan St vs Miami FL, team 2 wins
SCORE
North Carolina 

In [ ]:
teamTeamWinChanceMap